# Scrape Online Travel Agent (OTA) Property Prices

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [4]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Logging Configuration
___TODO___ add logging of status

In [2]:
import logging


## Initialize the OTA scrape class

In [5]:
import sys
sys.path.insert(1, '../../lib/wrangler')
import otaWebScraper as otaws
import datetime

dataDirPath="../../data/hospitality/bookings/scraper"

if debug:
    import importlib
    otaws = importlib.reload(otaws)
    
clsScraper = otaws.OTAWebScraper()

All packages in OTAWebScraper loaded successfully!


## Initialize the set of OTA URL for Scraping

In [21]:
from datetime import date

path = dataDirPath
file = "otaInputURLs.json"
start_date = date(2022,9,7)
end_date = date(2022,9,8)

try:
    if start_date < date.today():
        raise ValueError("Start date must be greater than today: %s" % str(date.today()))
    if end_date <= start_date:
        raise ValueError("End date %s is invalid. It must be greater than Start Date: %s" % (str(end_date),str(start_date)))
    scrape_crietia_dict = {"pageOffset":10,
                           "pageUpperLimit":550,
                           "startDate": start_date,
                           "endDate" : end_date,
             }
    _, _ota_url_parameterized_list  = clsScraper.build_scrape_url_list(dirPath=path,
                                                                       fileName=file,
                                                                       **scrape_crietia_dict)
except Exception as err:
    _s_fn_id = "Class <WebScraper> Function <read_folder_csv_to_df>"
    print("[Error]"+_s_fn_id, err)
    print(traceback.format_exc())

#print(_ota_url_parameterized_list)
print("Completed parameterizing urls with %d instances." % (len(_ota_url_parameterized_list)))

Loaded 1 properties to begin scraping OTA data.
Processing booking.com ...
Completed parameterizing urls with 784 instances.


In [22]:
#path = dataDirPath
file = "otaInputURLs.json"
property_dict = clsScraper.load_ota_list(dirPath=dataDirPath, fileName=file)
_scrape_tags_df = clsScraper.get_scrape_html_tags(property_dict)
print(_scrape_tags_df)

        variable   tag                   code          ota
0  content_block   div            .d20f4628d0  booking.com
0  property_name  span  fcab3ed991 a23c043802  booking.com
0      room_type  span             df597226dd  booking.com
0      room_rate   div  fcab3ed991 bd73d13072  booking.com
0   review_score   div  b5cd09854e d10a6220b4  booking.com
0       location   div             a1fbd102d9  booking.com
0          Other   div             d22a7c133b  booking.com


## Prepare folder & file structure

In [23]:
from datetime import datetime, timezone

dirPath = None
_search_dt = datetime.now()
kwargs = {'search_datetime': _search_dt}
''' include the timezone '''
_search_dt = (_search_dt.replace(tzinfo=timezone.utc)).isoformat()

_current_search_data_dir = clsScraper.get_search_data_dir_path(dirPath, **kwargs)
print("Extracting data into %s for search datetime: %s" % (_current_search_data_dir,str(_search_dt)))

Extracting data into ../../data/hospitality/bookings/scraper/rates/2022-9-6-9-30/ for search datetime: 2022-09-06T09:48:12.842229+00:00


## Scrape data into CSVs

In [24]:
_l_saved_files = clsScraper.scrape_url_list(_ota_url_parameterized_list, _search_dt, _current_search_data_dir)

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 561, in _scrape_bookings_to_csv
    _data_dict['Review Score'] = _list.find('div', class_='b5cd09854e d10a6220b4').text
AttributeError: 'NoneType' object has no attribute 'text'

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 561, in _scrape_bookings_to_csv
    _data_dict['Review Score'] = _list.find('div', class_='b5cd09854e d10a6220b4').text
AttributeError: 'NoneType' object has no attribute 'text'

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nu

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=280
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=350
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=420
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=490
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

...

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=210
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=280
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=350
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=420
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-07&checkout=2022-09-08&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=490
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hotel_bookings/../../lib/wrangler/otaWebScraper.py", line 553, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid

## Read from CSV to Dataframe

In [8]:
dirPath = _current_search_data_dir
#dirPath = "../../data/hospitality/bookings/scraper/rates/2022-9-6-9-30/"
removed_files = clsScraper.remove_empty_files(dirPath)
if len(removed_files) > 0:
    print("%d files removed" %len(removed_files))
kwargs = {"columns" : ["search_datetime",
                     "checkin_date",
                     "propery_name",
                     "room_type",
                     "room_rate",
                     "review_score",
                     "destination_id",
                     "other"]}

rates_df = clsScraper.read_folder_csv_to_df(dirPath, **kwargs)

In [9]:
for some in rates_df["room_type"]:
    print(some)

King Room - Ground Floor
Queen Room with Two Queen Beds - Hearing Accessible
Queen Room with Two Queen Beds - Non-Smoking
Queen Room with Two Queen Beds  - Hearing Accessible - Non-Smoking
Queen Room with Two Queen Beds
Queen Studio - Non-Smoking
Guest room, 2 Queen
Queen Room with Two Queen Beds - Hearing Access
King Leisure Non Smoking 
Queen Room
Executive King Suite City View with Sofabed,  Whirlpool In Bath/Non Smoking
King Room
King Room
King Room
Queen Room with 2 Queen Beds
Queen Studio with Two Queen Beds - Non-Smoking
Queen Room with Two Queen Beds -  Second Floor
Deluxe Double Room with Two Double Beds (Wide Bay)
Queen Studio
Queen Room with Two Queen Beds - Non-Smoking
Queen Room with Two Queen Beds with City View 
Bed in 6-Bed Mixed Dormitory Room
King Room - Non-Smoking
Standard Room with Two Double Beds Smoke Free
Standard Queen Room
Queen Room - Non-Smoking
Queen Studio - Non-Smoking
Deluxe Queen Room
Queen Studio - Disability Access/Non-Smoking
King Room with Sofa Bed 

In [28]:
print(rates_df.shape)
print(rates_df.columns)
print(rates_df.head(20))

(5098, 7)
Index(['search_datetime', 'propery_name', 'room_type', 'room_rate',
       'review_score', 'destination_id', 'other'],
      dtype='object')
                     search_datetime  \
0   2022-09-06T09:48:12.842229+00:00   
1   2022-09-06T09:48:12.842229+00:00   
2   2022-09-06T09:48:12.842229+00:00   
3   2022-09-06T09:48:12.842229+00:00   
4   2022-09-06T09:48:12.842229+00:00   
5   2022-09-06T09:48:12.842229+00:00   
6   2022-09-06T09:48:12.842229+00:00   
7   2022-09-06T09:48:12.842229+00:00   
8   2022-09-06T09:48:12.842229+00:00   
9   2022-09-06T09:48:12.842229+00:00   
10  2022-09-06T09:48:12.842229+00:00   
11  2022-09-06T09:48:12.842229+00:00   
12  2022-09-06T09:48:12.842229+00:00   
13  2022-09-06T09:48:12.842229+00:00   
14  2022-09-06T09:48:12.842229+00:00   
15  2022-09-06T09:48:12.842229+00:00   
16  2022-09-06T09:48:12.842229+00:00   
17  2022-09-06T09:48:12.842229+00:00   
18  2022-09-06T09:48:12.842229+00:00   
19  2022-09-06T09:48:12.842229+00:00   

        